In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from multiprocessing import Pool
import datetime

import scrapers

In [20]:
import itertools


def p_imap_unordered_set(pool, func, iterable):
    temp = pool.imap_unordered(func, iterable)
    return set(itertools.chain.from_iterable(temp))


the_time = lambda: datetime.datetime.now().time().strftime('%H:%M:%S')

In [23]:
riders_done = {"rider/"}
teams_done = {"team/"}
results = set()
teams_to_do = set()

print(the_time())

with Pool(50) as p:
    top_riders = p_imap_unordered_set(
        p, scrapers.scrape_top, itertools.product(range(1946, 2021), (range(0, 301, 100)))
    )

riders_to_do = top_riders.copy()
for i in range(50):
    if not riders_to_do:
        break
    print(f"{the_time()} - iter {i+1}, riders_to_do: {len(riders_to_do)}")
    with Pool(50) as p:
        fresh_results = p_imap_unordered_set(p, scrapers.scrape_rider, riders_to_do)

    results.update(fresh_results)
    riders_done.update(riders_to_do)

    teams_to_do = set(team for _, team, _ in fresh_results) - teams_done

    print(f"{the_time()} - teams_to_do: {len(teams_to_do)}")

    with Pool(50) as p:
        riders_to_do = p_imap_unordered_set(p, scrapers.scrape_team, teams_to_do) - riders_done

    teams_done.update(teams_to_do)

print(the_time())

16:43:54
16:44:08 - iter 1, riders_to_do: 6764
16:48:32 - teams_to_do: 5202
16:49:55 - iter 2, riders_to_do: 14398
16:58:37 - teams_to_do: 1209
16:59:00 - iter 3, riders_to_do: 4260
17:01:33 - teams_to_do: 162
17:01:42 - iter 4, riders_to_do: 777
17:02:18 - teams_to_do: 450
17:02:31 - iter 5, riders_to_do: 1592
17:03:34 - teams_to_do: 125
17:03:42 - iter 6, riders_to_do: 462
17:04:04 - teams_to_do: 7
17:04:07 - iter 7, riders_to_do: 17
17:04:11 - teams_to_do: 0
17:04:11


In [24]:
with open("top_riders.txt", "w") as f:
    f.write("\n".join(sorted(top_riders)))

df = pd.DataFrame(iter(results), columns=["rider", "team", "points"])
df.to_csv("scraped.csv", index=False)